In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py

--2025-07-03 05:50:20--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5820 (5.7K) [text/plain]
Saving to: ‘minsearch.py.1’

minsearch.py.1      100%[===================>]   5.68K  --.-KB/s    in 0s      

2025-07-03 05:50:20 (34.0 MB/s) - ‘minsearch.py.1’ saved [5820/5820]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
q = 'the course has already started, can I still enroll'

In [9]:
index.fit(documents)

In [40]:
boost = {'question' : 1.0, 'section': 1.0}

results = index.search(
    query=q,
    #filter_dict={'course':'llm-zoomcamp'},
    boost_dict=boost,
    num_results=5    
)    

In [41]:
results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequen

In [42]:
#from openai import OpenAI
#client = OpenAI()
from google import genai
client = genai.Client()

In [43]:
from google.genai import types

In [44]:
response =  client.models.generate_content(
    model="gemini-2.5-flash",
    contents = q,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking
    ),
)
print(response.text)

It depends on the institution and the specific course. Some might allow late enrollment, especially if you can quickly catch up. Others have strict deadlines once classes begin.

Here's what you should do:

1.  **Contact the Admissions Office or Course Administrator Immediately:** This is the most direct way to get an answer. Explain your situation and ask about their policy on late enrollment.
2.  **Be Prepared to Catch Up:** If they allow it, be ready to work extra hard to make up for missed material, assignments, and lectures. Ask about resources like recorded lectures or notes from classmates.
3.  **Inquire About Prorated Tuition/Fees:** If you enroll late, ask if there's any adjustment to the tuition or fees.
4.  **Consider Alternatives:** If late enrollment isn't possible, ask if there's an option to audit the course (attend without credit) or if there's an upcoming session you could join.

The sooner you ask, the better your chances!


In [45]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Restrict your answers to the QUESTION to the content in CONTEXT only.

QUESTION : {question}

CONTEXT:
{context}

"""

In [46]:
context = ""
for doc in results:
    content = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [47]:
prompt = prompt_template.format(question=q, context=content).strip()

In [48]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Restrict your answers to the QUESTION to the content in CONTEXT only.

QUESTION : the course has already started, can I still enroll

CONTEXT:
section: General course-related questions
question: Course - Can I get support if I take the course in the self-paced mode?
answer: Yes, the slack channel remains open and you can ask questions there. But always sDocker containers exit code w search the channel first and second, check the FAQ (this document), most likely all your questions are already answered here.
You can also tag the bot @ZoomcampQABot to help you conduct the search, but don’t rely on its answers 100%, it is pretty good though.


In [49]:
response =  client.models.generate_content(
    model="gemini-2.5-flash",
    contents = prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0) # Disables thinking
    ),
)
print(response.text)

Based on the provided context, there is no information available about course enrollment after the start date.
